In [1]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.

    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4
Mon Jun 19 14:44:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            O

In [2]:
!pip install pystemmer
!pip install optuna==2.3.0
# !pip install transformers #==4.2.1
!pip install tokenizers #==0.9.4
!pip install transformers==4.28.0
!pip install accelerate

In [3]:
import numpy as np
import pandas as pd


import re, functools, operator, string
import torch , optuna, gc, random, os

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
from sklearn.feature_selection import SelectPercentile , f_classif
import transformers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
data=pd.read_csv('/kaggle/input/hate-data/annot_mini_total_data.csv')

In [5]:

# column_titles = ['Tweet', 'label']  # Replace with your desired column titles
# data.columns = column_titles
data.head()

,Unnamed: 0,Tweet,Class,label,type
0,0,البطل قاتل وجاذف بحياته لتحيا انت واطي عيب الشوم,Abusive,Abusive,hate
1,1,انو غريب يوجعك راسك القواتجيه عاده بيكونو بلا راس,Racism,discrimination,hate
2,2,طاءفي روح اسال اهل قانا وهني بخبروك,Racism,discrimination,hate
3,3,غرد الجحش وظن باءنه حصانا تتكلم العوني بموضوعي...,Abusive,Abusive,hate
4,4,الفلسطينيين واللبنانين والشاميين لكان انتو للي...,Racism,discrimination,hate


# split normal data

In [6]:
#splitting data to train and test
# First setting the max_len , will be useful later for BERT Model
Extra_Len = 6 # an extra padding in length , found to be useful for increasing F-score
#Max_Len = data["Tweet"].str.split().str.len().max() + Extra_Len
for i in range (len(data["Tweet"])):
  if len(data["Tweet"][i].split())<512:
    data["Tweet"][i]=data["Tweet"][i]
  else:
    words = data["Tweet"][i].split()  # Split the string into a list of words
    truncated_words = words[:512]  # Take only the desired number of words
    data["Tweet"][i]= ' '.join(truncated_words)
data = data.dropna()
Max_Len=512
print(Max_Len)

#Spliting the Training data
Test_Size = 0.2
Rand_Seed = 333

train_set, test_set = train_test_split( data, test_size= Test_Size, random_state= Rand_Seed)

# print("Train set: ")
# print(train_set["label"].value_counts())
# print("---------------------------")
# print ("Test set: ")
# print (test_set["label"].value_counts())

/tmp/ipykernel_14319/2431392801.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Tweet"][i]=data["Tweet"][i]
/tmp/ipykernel_14319/2431392801.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Tweet"][i]= ' '.join(truncated_words)


512


In [7]:
train_set.head()

,Unnamed: 0,Tweet,Class,label,type
2192,2223,لاشكر واجب أختي الكريمة,normal,normal,normal
3556,3587,تعلم أصول اللغة والكتابة الصحيحة أولا الصعلوك ...,Abusive,Abusive,hate
6603,6635,بلاش تحكم علي حد و انت فيك و فيك انت ملكش غير ...,normal,normal,normal
1161,1186,لن ينصلح حال الشعب الفلسطيني وهذا الحمار الصهي...,Abusive,Abusive,hate
4972,5004,خاصكم داعش الكفار تفو العلمانيين,religion_off,discrimination,hate


In [8]:

# Step 2: Create an instance of the LabelEncoder
label_encoder = LabelEncoder()

# Step 3: Prepare your categorical labels
labels = data["label"]

# Step 4: Fit the LabelEncoder to your labels
label_encoder.fit(labels)

# Step 5: Encode the categorical labels into numerical values
labels = label_encoder.transform(labels)
data["labels"]=labels

In [9]:
data["labels"]

0       0
1       1
2       1
3       0
4       1
       ..
6950    2
6951    2
6952    2
6953    2
6954    2
Name: labels, Length: 6955, dtype: int64

In [10]:

X = data['Tweet']
y = data['labels']
df=pd.DataFrame(X)


In [11]:
VecModel = TfidfVectorizer()
X= VecModel.fit_transform(X)
print(f'The new shape for X is {X.shape}')

The new shape for X is (6955, 35975)


In [12]:
FeatureSelection = SelectPercentile(score_func = f_classif, percentile=1)
X = FeatureSelection.fit_transform(X, y)

In [13]:
print('X Shape is ' , X.shape)

X Shape is  (6955, 360)


In [14]:
X

<6955x360 sparse matrix of type '<class 'numpy.float64'>'
	with 14760 stored elements in Compressed Sparse Row format>

In [15]:
train_set, test_set = train_test_split( data, test_size= Test_Size, random_state= Rand_Seed)

print("Train set: ")
print(train_set["labels"].value_counts())
print("---------------------------")
print ("Test set: ")
print (test_set["labels"].value_counts())

Train set: 
2    2821
0    1650
1    1093
Name: labels, dtype: int64
---------------------------
Test set: 
2    731
0    377
1    283
Name: labels, dtype: int64


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44, shuffle =True)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

X_train shape is  (5564, 360)
X_test shape is  (1391, 360)
y_train shape is  (5564,)
y_test shape is  (1391,)


# bert model intialisation


In [17]:
# a class representing the dataset
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [18]:
class BERTModelDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTModelDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name,ignore_mismatched_sizes=True)
      self.max_len = max_len
      self.label_map = label_map

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      encoded_review = self.tokenizer.encode_plus(
      text,
      max_length= self.max_len,
      add_special_tokens= True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      truncation='longest_first',
      return_attention_mask=True,
      return_tensors='pt'
    )
      input_ids = encoded_review['input_ids'].to(device)
      attention_mask = encoded_review['attention_mask'].to(device)

      return InputFeatures(input_ids=input_ids.flatten(), attention_mask=attention_mask.flatten(), label=self.label_map[self.target[item]])

In [19]:
#Defining Needed Methods for training and evaluation
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(Model_Used, return_dict=True,ignore_mismatched_sizes=True, num_labels=len(label_map))

def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [20]:
Model_Used = "UBC-NLP/MARBERT"
Task_Name = "classification"
Max_Len = 512
Output_File = "sub_filew.csv"
#define training arguments
training_args = transformers.TrainingArguments(
'marbert_multi20_class',
save_strategy="epoch",
save_total_limit=5,
# push_to_hub_model_type="pt",
lr_scheduler_type='cosine',
# evaluate_during_training=evaluate_during_training,
adam_epsilon=1e-8,
learning_rate=1.215e-05,
fp16=True,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
gradient_accumulation_steps=2,
num_train_epochs=20,
warmup_steps=0,
evaluation_strategy='epoch',
seed=42,
disable_tqdm=False,
dataloader_pin_memory=False,

)





In [21]:
label_list = list(train_set["labels"].unique())

print(label_list)
print(train_set["labels"].value_counts())

data_set = Dataset( "hateclass", train_set, test_set, label_list )

label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

train_dataset = BERTModelDataset(train_set["Tweet"].to_list(),
                                 train_set["labels"].to_list(),Model_Used,int(Max_Len),label_map)

test_dataset = BERTModelDataset(test_set["Tweet"].to_list(),
                                      test_set["labels"].to_list(),Model_Used,int(Max_Len),label_map)

[2, 0, 1]
2    2821
0    1650
1    1093
Name: labels, dtype: int64
{2: 0, 0: 1, 1: 2}


In [22]:
training_args.dataloader_pin_memory = False
gc.collect()
torch.cuda.empty_cache()
set_seed(Rand_Seed)

trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset= test_dataset,
    compute_metrics=compute_metrics
)

print(training_args.seed)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

42


In [26]:
!rm -rf /kaggle/working/wandb

In [ ]:
import os
gc.collect()
torch.cuda.empty_cache()

os.environ["WANDB_DISABLED"] = "true"
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy
1,No log,0.555381,0.699259,0.706562,0.693713,0.754134
2,No log,0.571081,0.743473,0.735609,0.765703,0.767793
3,0.492400,0.571303,0.768553,0.762978,0.776036,0.798706
4,0.492400,0.751510,0.759906,0.756025,0.775352,0.785766
5,0.492400,0.917307,0.775717,0.770632,0.782099,0.808052


              precision    recall  f1-score   support

           0       0.84      0.89      0.87       731
           1       0.67      0.64      0.65       377
           2       0.61      0.55      0.58       283

    accuracy                           0.75      1391
   macro avg       0.71      0.69      0.70      1391
weighted avg       0.75      0.75      0.75      1391



/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.94      0.78      0.85       731
           1       0.68      0.74      0.71       377
           2       0.59      0.78      0.67       283

    accuracy                           0.77      1391
   macro avg       0.74      0.77      0.74      1391
weighted avg       0.80      0.77      0.78      1391



...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.94      0.80      0.86       731
           1       0.65      0.83      0.73       377
           2       0.67      0.70      0.69       283

    accuracy                           0.79      1391
   macro avg       0.76      0.78      0.76      1391
weighted avg       0.81      0.79      0.79      1391



/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.92      0.87      0.89       731
           1       0.72      0.77      0.74       377
           2       0.67      0.71      0.69       283

    accuracy                           0.81      1391
   macro avg       0.77      0.78      0.78      1391
weighted avg       0.81      0.81      0.81      1391



/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


## aubmindlab/bert-base-arabertv2

In [ ]:
Model_Used = "lamaabdulaziz/AraBERT-finetuned-CrossVal-fnd"
Task_Name = "classification"
Max_Len = 512
Output_File = "sub_filew.csv"
#define training arguments

training_args = transformers.TrainingArguments(
'arabert',
save_strategy="epoch",
save_total_limit=5,
# push_to_hub_model_type="pt",
lr_scheduler_type='cosine',
# evaluate_during_training=evaluate_during_training,
adam_epsilon=1e-8,
learning_rate=2e-05,
fp16=True,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
gradient_accumulation_steps=2,
num_train_epochs=4,
warmup_steps=0,
evaluation_strategy='epoch',
seed=42,
disable_tqdm=False,
dataloader_pin_memory=False,

)


In [ ]:
label_list = list(train_set["label"].unique())

print(label_list)
print(train_set["label"].value_counts())

data_set = Dataset( "3hsd", train_set, test_set, label_list )

label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

train_dataset = BERTModelDataset(train_set["Tweet"].to_list(),
                                 train_set["label"].to_list(),Model_Used,int(Max_Len),label_map)

test_dataset = BERTModelDataset(test_set["Tweet"].to_list(),
                                      test_set["label"].to_list(),Model_Used,int(Max_Len),label_map)

In [ ]:
training_args.dataloader_pin_memory = False
gc.collect()
torch.cuda.empty_cache()
set_seed(Rand_Seed)

trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset= test_dataset,
    compute_metrics=compute_metrics
)

print(training_args.seed)

In [ ]:
import os
gc.collect()
torch.cuda.empty_cache()

os.environ["WANDB_DISABLED"] = "true"
trainer.train()

# camel_bert

In [ ]:
Model_Used = "CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment"
Task_Name = "classification"
Max_Len = 512

#define training arguments

training_args = transformers.TrainingArguments(
'camel_mix_sentiment',
save_strategy="epoch",
save_total_limit=5,
# push_to_hub_model_type="pt",
lr_scheduler_type='cosine',
# evaluate_during_training=evaluate_during_training,
adam_epsilon=1e-8,
learning_rate=2e-05,
fp16=True,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
gradient_accumulation_steps=2,
num_train_epochs=4,
warmup_steps=0,
evaluation_strategy='epoch',
seed=42,
disable_tqdm=False,
dataloader_pin_memory=False,

)


In [ ]:
label_list = list(train_set["label"].unique())

print(label_list)
print(train_set["label"].value_counts())

data_set = Dataset( "3hsd", train_set, test_set, label_list )

label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

train_dataset = BERTModelDataset(train_set["Tweet"].to_list(),
                                 train_set["label"].to_list(),Model_Used,int(Max_Len),label_map)

test_dataset = BERTModelDataset(test_set["Tweet"].to_list(),
                                      test_set["label"].to_list(),Model_Used,int(Max_Len),label_map)

In [ ]:
training_args.dataloader_pin_memory = False
gc.collect()
torch.cuda.empty_cache()
set_seed(Rand_Seed)

trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset= test_dataset,
    compute_metrics=compute_metrics
)

print(training_args.seed)

In [ ]:
import os
gc.collect()
torch.cuda.empty_cache()

os.environ["WANDB_DISABLED"] = "true"
trainer.train()